<a href="https://colab.research.google.com/github/prabhkaur119/notebooks/blob/main/stable_video_diffusion_fp16_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade transformers

In [2]:
import torch
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline
from PIL import Image
import os

In [3]:
# Create directory for images
os.makedirs("/content/sdxl_images", exist_ok=True)

In [ ]:
# Load base model
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

In [5]:
# Enable memory optimizations
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

In [ ]:
# Modified pipeline with realism focus
realistic_pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16
).to("cuda")

# Hyperparameters for realism
realism_prompt = "photorealistic, 8K UHD, DSLR, high detail, " + prompt
negative_prompt = "cartoon, 3d, digital art, anime, painting"

image = realistic_pipe(
    prompt=realism_prompt,
    negative_prompt=negative_prompt,
    width=1024 if has_memory else 768,
    num_inference_steps=40,
    guidance_scale=8.5
).images[0]

In [ ]:
# Your prompts (first 3 for demo - remove [0:3] to generate more)
prompts = ["A dragon sleeping on a pile of golden coins inside a cave", "A fairy queen in a glowing forest at night", "A medieval knight riding a unicorn through the mist", "A wizard casting a spell with glowing runes around him", "An enchanted castle floating in the sky", "A mermaid sitting on a rock during a stormy sunset", "A phoenix rising from ashes in a dark forest", "A battle between gods on top of a mountain", "A dwarven blacksmith forging a glowing magical sword", "A portal opening in the middle of a peaceful village", "A cyberpunk cityscape at night with neon lights and flying cars", "A humanoid robot walking through a deserted alien planet", "A space station orbiting Saturn", "An astronaut discovering an ancient alien artifact", "A futuristic soldier in an exosuit overlooking a battlefield", "A city on Mars with domes and red desert in the background", "A post-apocalyptic street filled with overgrown plants", "A spaceship docking at a high-tech interstellar hub", "A holographic teacher in a virtual classroom", "A biotech laboratory filled with glowing blue liquid tanks", "A sunrise over misty mountains and a serene lake", "A dense jungle with ancient ruins hidden inside", "A field of lavender under a starry sky", "A desert canyon during golden hour", "A snowy forest with northern lights above", "A tropical beach with turquoise waters and palm trees", "A quiet waterfall deep in the forest", "A volcanic eruption seen from a nearby village", "A rice terrace farm at sunset in Southeast Asia", "A storm approaching over an empty prairie", "A staircase leading to the clouds in a blue sky", "A whale floating above a city skyline", "A man with a clock for a face sitting in a desert", "A city made entirely of glass and light", "A forest where trees grow upside down", "A dreamscape where the ocean meets the stars", "A girl holding an umbrella raining fish", "A landscape where time stands still and objects float", "A bridge connecting two floating islands in space", "A surreal carnival with melting rides", "A steampunk inventor with mechanical arms", "A cyberpunk girl with neon tattoos and LED hair", "A samurai warrior under cherry blossoms", "A pirate queen with a mechanical parrot", "A Victorian lady with a floral hat and soft expression", "A cowboy walking into a dusty sunset", "A vampire aristocrat in a gothic mansion", "An alien with translucent skin and glowing eyes", "A medieval healer holding herbs in a forest", "A warrior monk with a staff standing in a temple", "A futuristic skyscraper with organic shapes and glowing windows", "An abandoned gothic cathedral overgrown with ivy", "A medieval town square during a festival", "A Venetian canal at dusk with reflections in the water", "A Moroccan marketplace bustling with color", "A Nordic village by the fjords during winter", "A giant tree with houses built in its branches", "An art deco building in the middle of a retro-futuristic city", "A Greek temple on a cliff above the sea", "A utopian city floating in the sky", "A white tiger walking through a bamboo forest", "A golden retriever astronaut on the moon", "A chameleon blending into a rainbow jungle", "An owl with galaxy patterns on its feathers", "A mechanical wolf with glowing red eyes", "A family of foxes playing under falling leaves", "A jellyfish with bioluminescent tentacles in deep sea", "A peacock spreading feathers made of stars", "A horse galloping through a thunderstorm", "A panda sitting in a zen garden", "An ancient Egyptian priest standing beside a pyramid", "A Roman centurion preparing for battle", "A traditional Japanese tea ceremony in a wooden room", "A Viking ship sailing under a red sky", "A Renaissance painter working in a candle-lit studio", "A traditional Indian wedding procession with elephants", "A Native American dancer in ceremonial attire", "A medieval market with traders and musicians", "An old Chinese scholar writing poetry by candlelight", "A jazz band playing in a 1920s speakeasy"]

# Negative prompt to avoid common issues
negative_prompt = "blurry, deformed, extra limbs, text, watermark, low quality"

for i, prompt in enumerate(prompts):
    print(f"Generating image {i+1}/{len(prompts)}: {prompt[:50]}...")

    try:
        # Generate image with reduced memory footprint
        image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            width=768,  # Reduced from 1024 to save memory
            height=768,
            num_inference_steps=25,  # Reduced from 30
            guidance_scale=7.5  # Standard value
        ).images[0]
        # Save image
        image.save(f"/content/sdxl_images/image_{i}.png")
        print(f"Successfully generated image {i+1}")

        # Clear memory
        torch.cuda.empty_cache()

    except Exception as e:
        print(f"Failed to generate image {i+1}: {str(e)}")
        continue

# Display first image if available
if os.path.exists("/content/sdxl_images/image_0.png"):
    display(Image.open("/content/sdxl_images/image_0.png"))

# Zip all images for download
!zip -r /content/sdxl_images.zip /content/sdxl_images

print("\nDone! Download your images:")
from google.colab import files
files.download("/content/sdxl_images.zip")